In [1]:
import pandas as pd
import numpy as np
import re
import glob

In [2]:
class result:
    def __init__(self, ID, authors, year, journal, title, abstract, keywords, decision, no, pNo, yes, pYes, note, path):
        self.ID = ID
        self.authors = str(authors)
        self.year = year
        self.journal = str(journal)
        self.title = str(title)
        self.abstract = str(abstract)
        self.keywords = str(keywords)
        self.decision =decision
        self.no = str(no)
        if(self.no =='nan'):
            self.no=""
        self.pNo = str(pNo)
        if(self.pNo =='nan'):
            self.pNo=""
            
        self.yes = str(yes)
        if(self.yes =='nan'):
            self.yes=""
            
        self.pYes = str(pYes)
        if(self.pYes =='nan'):
            self.pYes=""
            
        self.note = str(note)
        if(self.note =='nan'):
            self.note=""
        self.path = str(path)
        
paths = glob.glob("*.csv")##saving all screening results as instances of result
refList = []
for path in paths:
    train = pd.read_csv(path); #change it into your file name if it is in other names
    for i in range(len(train['ID'])):
        #j = str(result['Journal'])
        refList.append(result(train.loc[i][0], train.loc[i][1],train.loc[i][2],train.loc[i][3],train.loc[i][4],train.loc[i][5],train.loc[i][6],train.loc[i][7],train.loc[i][8],train.loc[i][9],train.loc[i][10],train.loc[i][11],train.loc[i][12], path))
        
            

In [3]:
def writeTo():#for writing back to an excel spreadsheet
    if(result.ID not in file['ID']): 
        file['ID'].append(result.ID)
        file['Authors'].append(result.authors)
        file['Year'].append(result.year)
        file['Journal'].append(result.journal)
        file['Title'].append(result.title)
        file['Abstract'].append(result.abstract)
        file['Keywords'].append(result.keywords)
        file['Decision'].append(result.decision)
        file['Terms for No'].append(result.no)
        file['Terms for P No'].append(result.pNo) 
        file['Terms for P Yes'].append(result.pYes)  
        file['Terms for YES'].append(result.yes) 
        file['Extra note'].append(result.note) 
                


In [4]:
def concat(a,b):
    
    if (a =="" and b!=""):
        return b
    elif(a !="" and b==""):
        return a
    elif(a =="" and b==""):
        return ""
    else:
        return a +", "+b

def duplicates():#de-duplication
    if result.decision==-2 and compare.decision==-1:
        result.decision = -1
        result.note = "*Merged to -1. " + concat(result.note,compare.note)
    elif result.decision==-1 and compare.decision==-2:
        result.decision = -1
        result.note = "*Merged to -1. " + concat(result.note,compare.note)
    elif result.decision==2 and compare.decision==1:
        result.decision = 1
        result.note = "*Merged to 1. " + concat(result.note,compare.note)
    elif result.decision==1 and compare.decision==2:
        result.decision = 1
        result.note = "*Merged to 1. " + concat(result.note,compare.note)
        
    elif result.decision == compare.decision:
        
        result.note = "*Merged, no conflict. " + concat(result.note,compare.note)
    elif result.decision != compare.decision:
        result.note = "*Judgement needed. ("+str(result.decision)+str(compare.decision)+") " + concat(result.note,compare.note)
    else:
        print('no decision found' + str(result.decision) + ' and '+str(compare.decision))
    
    
    result.yes = concat(result.yes,compare.yes)
    result.no = concat(result.no,compare.no)
    result.pYes = concat(result.pYes,compare.pYes)
    result.pNo = concat(result.pNo,compare.pNo)
    
###############contradictions in reviewer decisions
d = {'ID': [], 'Path 1': [], 'Decision 1': [], 'Decision 2': [], 'Path 2': [],  'Abstract':[]}
file = {'ID': [],'Authors': [],'Year': [],'Journal': [],'Title': [],'Abstract': [], 'Keywords': [],'Decision': [],'Terms for No': [],'Terms for P No': [],'Terms for P Yes': [],
        'Terms for YES': [],'Extra note': [],}


for result in refList:#for each screening result
    
    for compare in refList:#for each screening result: going through the list again, to compair elements pairwise
        if (result.ID == compare.ID) and (result.path != compare.path):#pairwise comparison found matching IDs with a different result(not with itself since path is different)
            #print('Id a = %s, IDb = %s'%(result.path, compare.path))
            
            
            
            if result.decision != compare.decision:#we only care if the decisions are different in those 2 results. Otherwise, data is written back to the master file
                if(result.ID not in d['ID']):
                    #############df to display results
                    d['ID'].append(result.ID)
                    d['Path 1'].append(result.path)
                    d['Path 2'].append(compare.path)
                    d['Decision 1'].append(result.decision)
                    d['Decision 2'].append(compare.decision)
                    d['Abstract'].append(result.abstract)
                    #delete both disagreeing results
                    #assimilate the close results and make them 1
                    #add the revised results
            duplicates()        
    writeTo() 
                
        
    
                    
                    


In [5]:
def addDict(dict1, line):
    words =re.split(r'[;,\r\n]', line)
    for w in words:
        w= re.sub(r'\*', '', w)#removing unwanted differences
        w=w.lower().strip()    
        if (w != '' and w !="nan"):#exclude those
            
            dict1.get(w)#removing unwanted differences
            dict1[w] = dict1.get(w, 0) + 1#the dict that counts occurrences by field
            
            if (w in dict_all): #increment counting variable
                dict_all[w][0] += 1
        
            else: #make entry vector
                dict_all[w] = [1,0,0, 0, 0]
            if(result.decision == 2):
                dict_all[w][1] += 1#yes
            elif(result.decision == 1): 
                dict_all[w][2] += 1#pyes
            elif(result.decision == -1):
                dict_all[w][3] += 1#pno
            elif(result.decision == -2):
                dict_all[w][4] += 1#no
            else:
                print('error ' + str(result.decision))

def addKeywords(line):
    words = re.split(r'[;,\r\n]', line)#####keeps the wildcard stars
    for w in words:
        w=w.lower().strip()
        if (w != '' and w !="nan"):
            if (w in dict_keywords): #increment counting variable
                dict_keywords[w][0] += 1
        
            else: #make entry vector
                dict_keywords[w] = [1,0,0, 0, 0]
            if(result.decision == 2):
                dict_keywords[w][1] += 1#yes
            elif(result.decision == 1): 
                dict_keywords[w][2] += 1#pyes
            elif(result.decision == -1):
                dict_keywords[w][3] += 1#pno
            elif(result.decision == -2):
                dict_keywords[w][4] += 1#no
            #else:
                #print('error ' + str(result.decision))
            
    

In [6]:
###########de-ducplicating the terms givien by reviewers and making dicts for excel file

dict_no = {}
dict_Pno = {}
dict_Pyes = {}
dict_yes = {}
dict_all = {}

for result in refList:##for custom terms
    
    addDict(dict_yes, result.yes)
    addDict(dict_no, result.no)
    addDict(dict_Pno, result.pNo)
    addDict(dict_Pyes, result.pYes)
    
words = {'Word': [], 'Total': [], 'Yes': [], 'P yes': [], 'P no': [],  'No':[]}#for excel
for key,value in dict_all.items():
    words['Word'].append(key)
    words['Total'].append(dict_all[key][0])
    words['Yes'].append(dict_all[key][1])
    words['P yes'].append(dict_all[key][2])
    words['P no'].append(dict_all[key][3])
    words['No'].append(dict_all[key][4]) 
    
dict_keywords = {}
for result in refList:
    addKeywords(result.keywords)
keywords = {'Word': [], 'Total': [], 'Yes': [], 'P yes': [], 'P no': [],  'No':[]}#for excel
for key,value in dict_keywords.items():
    keywords['Word'].append(key)
    keywords['Total'].append(dict_keywords[key][0])
    keywords['Yes'].append(dict_keywords[key][1])
    keywords['P yes'].append(dict_keywords[key][2])
    keywords['P no'].append(dict_keywords[key][3])
    keywords['No'].append(dict_keywords[key][4])     

error nan
error nan
error nan
error nan
error nan
error nan
error nan
error nan
error nan
error nan
error nan
error nan
error nan
error nan
error nan


In [7]:
print(len(dict_yes.keys()))
print(dict_yes)

167
{'natural language processing': 13, 'word embeddings': 2, 'automated content analysis': 2, 'quality indicators': 2, 'networks validation': 2, 'markov logic network': 2, 'causal association rules': 2, 'decision tree': 4, 'connectedness testing': 2, 'disconnected network': 2, 'graph theory': 2, 'indirect comparison': 2, 'network meta-analysis': 2, 'big data': 5, 'statistical analysis': 2, 'infoepidemiology': 2, 'infographics': 2, 'pattern recognition': 4, 'systematic reviews': 4, 'common data models': 2, 'cdm': 2, 'data harmonisation': 2, 'data reuse': 2, 'data transformation': 2, 'feature extraction': 2, 'meta-analysis': 9, 'reproducibility': 2, 'research methodologies': 2, 'automated text classification': 3, 'systematic review': 6, 'decision trees algorithm': 3, 'bibliographic databases': 2, 'information science': 2, 'data accuracy': 2, 'data collection': 2, 'screening': 5, 'study design': 3, 'automation of systematic reviews': 3, 'systematic reviews as topic': 2, 'data interpretat

In [8]:
#############ambivalent terms
set_no = set(dict_no.keys())


set_Pno = set(dict_Pno.keys())
set_Pyes = set(dict_Pyes.keys())
set_yes = set(dict_yes.keys())

no_Pno = set_no.intersection(set_Pno)#common members between no and pno
set_Pno = set_Pno-no_Pno#deleting the common members from pno, since they are already in no

yes_Pyes = set_yes.intersection(set_Pyes)#same for yes
set_Pyes = set_Pyes - yes_Pyes


yes_Pno = set_yes.intersection(set_Pno)
print("Appear in yes AND Pno "+ str(yes_Pno))
yes_no = set_yes.intersection(set_no)
print("Appear in yes AND no "+str(yes_no))
no_Pyes = set_no.intersection(set_Pyes)
print("Appear in no AND Pyes "+str(no_Pyes))



Appear in yes AND Pno {'information science', 'automated content analysis', 'quality indicators', 'markov logic network', 'algorithm', 'data mining', 'algorithms', 'bibliographic databases'}
Appear in yes AND no {'big data', 'statistical analysis', 'automation', 'network meta-analysis', 'common data models', 'systematic review', 'feature extraction', 'data reuse', 'natural language processing'}
Appear in no AND Pyes {'artificial intelligence', 'decision support systems'}


In [9]:
##############Relationship between journal and rating
journals = {}

#print(len(refList))
for result in refList:
    #print(result.journal)
    if (result.journal in journals): #increment counting variable
        journals[result.journal][0] += 1
        
    else: #make entry vector
        journals[result.journal] = [1,0,0,0,0]
    if(result.decision == 2):
        journals[result.journal][1] += 1#yes
    elif(result.decision == 1): 
        journals[result.journal][2] += 1#pyes
    elif(result.decision == -1):
        journals[result.journal][3] += 1#pno
    elif(result.decision == -2):
        journals[result.journal][4] += 1#no
    #else:
        #print('error ' + str(result.decision))
        
j = {'Name': [], 'Total': [], 'Yes': [], 'P yes': [], 'P no': [],  'No':[]}
for key,value in journals.items():
    j['Name'].append(key)
    j['Total'].append(journals[key][0])
    j['Yes'].append(journals[key][1])
    j['P yes'].append(journals[key][2])
    j['P no'].append(journals[key][3])
    j['No'].append(journals[key][4])

print(journals)    
df_journal = pd.DataFrame(data=j)

df_journal


{'Journal of clinical epidemiology': [24, 9, 4, 1, 10], 'PloS one': [166, 4, 9, 27, 124], 'Journal of the American Medical Informatics Association : JAMIA': [126, 6, 9, 18, 93], 'BMC medical research methodology': [14, 0, 2, 3, 9], 'PLoS medicine': [7, 0, 0, 2, 5], 'Artificial intelligence in medicine': [57, 1, 2, 15, 39], 'Systematic reviews': [47, 15, 9, 9, 14], 'Journal of medical Internet research': [64, 2, 0, 13, 48], 'Research synthesis methods': [37, 6, 13, 8, 9], 'Studies in health technology and informatics': [151, 5, 15, 31, 99], 'International journal of medical informatics': [74, 1, 4, 13, 56], 'BMC health services research': [15, 0, 0, 3, 12], 'BMC bioinformatics': [51, 2, 4, 7, 38], 'International journal of technology assessment in health care': [5, 0, 0, 0, 4], 'Journal of machine learning research : JMLR': [2, 2, 0, 0, 0]}


,Name,Total,Yes,P yes,P no,No
0,Journal of clinical epidemiology,24,9,4,1,10
1,PloS one,166,4,9,27,124
2,Journal of the American Medical Informatics As...,126,6,9,18,93
3,BMC medical research methodology,14,0,2,3,9
4,PLoS medicine,7,0,0,2,5
5,Artificial intelligence in medicine,57,1,2,15,39
6,Systematic reviews,47,15,9,9,14
7,Journal of medical Internet research,64,2,0,13,48
8,Research synthesis methods,37,6,13,8,9
9,Studies in health technology and informatics,151,5,15,31,99


In [10]:
###########relationship between year and rating

years = {}
#print(len(refList))
for result in refList:
    #print(result.journal)
    if (result.year in years): #increment counting variable
        years[result.year][0] += 1
        
    else: #make entry vector
        years[result.year] = [1,0,0,0,0]
    if(result.decision == 2):
        years[result.year][1] += 1#yes
    elif(result.decision == 1): 
        years[result.year][2] += 1#pyes
    elif(result.decision == -1):
        years[result.year][3] += 1#pno
    elif(result.decision == -2):
        years[result.year][4] += 1#no
    #else:
        #print('error ' + str(result.decision))
        
y = {'Year': [], 'Total': [], 'Yes': [], 'P yes': [], 'P no': [],  'No':[]}
for key,value in years.items():
    y['Year'].append(key)
    y['Total'].append(years[key][0])
    y['Yes'].append(years[key][1])
    y['P yes'].append(years[key][2])
    y['P no'].append(years[key][3])
    y['No'].append(years[key][4])        

print(years)    
df_years = pd.DataFrame(data=y)

df_years

{2018: [167, 14, 14, 60, 79], 2019: [2, 2, 0, 0, 0], 2011: [32, 2, 1, 8, 21], 2012: [34, 1, 2, 3, 28], 2013: [52, 3, 9, 1, 39], 2010: [32, 4, 2, 5, 21], 2009: [24, 2, 7, 2, 13], 2008: [23, 0, 0, 3, 20], 2007: [21, 0, 2, 0, 19], 2006: [14, 0, 2, 0, 12], 2001: [5, 0, 0, 2, 3], 2000: [7, 0, 0, 2, 5], 1999: [10, 0, 1, 4, 5], 1998: [10, 0, 0, 1, 9], 1997: [14, 0, 2, 0, 12], 1996: [3, 0, 0, 1, 2], 1995: [2, 0, 0, 0, 2], 1994: [6, 0, 0, 0, 6], 1993: [4, 0, 0, 2, 2], 2017: [175, 2, 12, 30, 131], 2016: [78, 7, 8, 6, 55], 2015: [57, 10, 6, 9, 29], 2014: [68, 6, 3, 11, 47]}


,Year,Total,Yes,P yes,P no,No
0,2018,167,14,14,60,79
1,2019,2,2,0,0,0
2,2011,32,2,1,8,21
3,2012,34,1,2,3,28
4,2013,52,3,9,1,39
5,2010,32,4,2,5,21
6,2009,24,2,7,2,13
7,2008,23,0,0,3,20
8,2007,21,0,2,0,19
9,2006,14,0,2,0,12


In [11]:
differenceFrame = pd.DataFrame(data=d)
masterFrame = pd.DataFrame(data=file)
journalFrame = pd.DataFrame(data=j)
yearFrame = pd.DataFrame(data=y)
wordsFrame = pd.DataFrame(data=words)
keywordsFrame = pd.DataFrame(data=keywords)
writer = pd.ExcelWriter('AutoSession2_results.xlsx', engine='xlsxwriter')
differenceFrame.to_excel(writer, sheet_name= 'Differences')
masterFrame.to_excel(writer, sheet_name= 'All')
journalFrame.to_excel(writer, sheet_name= 'Journal data')
yearFrame.to_excel(writer, sheet_name= 'Year data')
wordsFrame.to_excel(writer, sheet_name= 'Terms data')
keywordsFrame.to_excel(writer, sheet_name='Keywords data')
writer.save()               
        